In [ ]:
import JSSP
import gym
import time

In [ ]:

env_name = "JSSP-v0"
data = [[2,2],[2,1,0,5,1,1,10],[2,1,1,3,1,0,1]]
env = gym.make(env_name, env_data = data)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

In [ ]:
state = env.reset()
env.render(mode="human")
time.sleep(20)

# episodes = 10
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         #env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))